In [1]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer
import warnings

warnings.filterwarnings('ignore')

# ==============================================================================
# [Step 1] 데이터 병합 및 시드 데이터 준비
# ==============================================================================
print("🚀 [Step 1] 데이터 병합 및 시드 데이터셋 구성 중...")

# 1. 파일 로드
FILE_MONITOR = 'FlowScore_3.0_Monitoring_Report.csv' # 기초 지표
FILE_FEATURE = 'Flow_Engineered_Features.csv'        # 고급 피처 (방금 생성한 파일)

if not (os.path.exists(FILE_MONITOR) and os.path.exists(FILE_FEATURE)):
    print(f"❌ 파일이 없습니다. 경로를 확인해주세요.\n - {FILE_MONITOR}\n - {FILE_FEATURE}")
    exit()

df_base = pd.read_csv(FILE_MONITOR)
df_feat = pd.read_csv(FILE_FEATURE)

# 2. 병합
df_real = pd.merge(df_base, df_feat, on='Company_ID', how='inner')
print(f"✅ 병합 완료: 총 {len(df_real)}개 기업 데이터 확보")

# 3. 전처리 (숫자 변환)
features = [
    'Current_Ratio', 'Debt_Ratio', 'Late_Pay_Ratio', 'Avg_Delay_Days', 'Biz_Score', 'CEO_Score',
    'Sales_Growth', 'OPM_Change', 'Rev_Per_Emp', 'Tx_Volatility', 'Weekend_Tx_Ratio', 'Avg_Tx_Hour', 'Emp_Momentum'
]

def clean_num(x):
    if isinstance(x, str): return float(x.replace('%','').replace('일','').replace(',','').replace('-','0'))
    return float(x) if pd.notnull(x) else 0

for col in features:
    if col in df_real.columns:
        df_real[col] = df_real[col].apply(clean_num)
    else:
        df_real[col] = 0

# 4. CTGAN 학습용 '규칙 기반 거절(Seed Reject)' 데이터 생성
# (CTGAN에게 '이게 부실이야'라고 알려줄 초기 데이터)
def generate_seed_rejects(n):
    data = []
    for _ in range(n):
        row = {}
        # 무작위 기본값
        for col in features:
            if 'Ratio' in col: row[col] = np.random.uniform(0.1, 0.5)
            elif 'Score' in col: row[col] = np.random.randint(60, 80)
            else: row[col] = 0

        # 시나리오별 악화 (Perturbation)
        scenario = np.random.choice(['FIN', 'ACT'], p=[0.5, 0.5])
        
        if scenario == 'FIN': # 재무 부실
            row['Current_Ratio'] = np.random.uniform(0.2, 0.8)
            row['Debt_Ratio'] = np.random.uniform(400, 1000)
            row['Sales_Growth'] = np.random.uniform(-0.5, -0.1) # 역성장
        else: # 활동 부실
            row['Late_Pay_Ratio'] = np.random.uniform(0.4, 0.9)
            row['Tx_Volatility'] = np.random.uniform(0.5, 2.0) # 극심한 변동
            row['Avg_Tx_Hour'] = np.random.uniform(16, 22) # 늦은 결제

        row['Grade'] = 'Reject'
        data.append(row)
    return pd.DataFrame(data)

df_rule_rejects = generate_seed_rejects(500)
df_real['Grade'] = 'Approve'

# 통합 시드 데이터
df_seed_for_ctgan = pd.concat([df_real[features + ['Grade']], df_rule_rejects], ignore_index=True)

# ==============================================================================
# [Step 2] CTGAN 학습 및 고도화 데이터 생성
# ==============================================================================
print("\n🚀 [Step 2] CTGAN 학습 시작 (AI가 부실 패턴을 학습합니다)...")

# 1. 메타데이터 설정
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df_seed_for_ctgan)

# 2. 모델 학습
synthesizer = CTGANSynthesizer(metadata, epochs=300, verbose=True)
synthesizer.fit(df_seed_for_ctgan)

print("✅ CTGAN 학습 완료!")

# 3. 데이터 생성 (Sampling)
print("🏭 AI가 새로운 부실 기업 1000건 생성 중...")
df_ctgan_generated = synthesizer.sample(num_rows=1000)
df_ctgan_rejects = df_ctgan_generated[df_ctgan_generated['Grade'] == 'Reject'].copy()

# ==============================================================================
# [Step 3] 최종 모델 학습 (Random Forest)
# ==============================================================================
print("\n🚀 [Step 3] FlowScore 4.0 최종 모델 학습...")

# 데이터 조립 (실제 승인 + CTGAN 거절)
df_final_train = pd.concat([df_real[features + ['Grade']], df_ctgan_rejects], ignore_index=True)

X = df_final_train[features]
y = df_final_train['Grade'].apply(lambda x: 1 if 'Approve' in x else 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=200, max_depth=12, random_state=42)
model.fit(X_train, y_train)

# 평가
acc = accuracy_score(y_test, model.predict(X_test))
print(f"✅ 최종 모델 정확도: {acc*100:.2f}%")

# 중요도 확인
imp = pd.DataFrame({'Feature': features, 'Importance': model.feature_importances_}).sort_values('Importance', ascending=False)
print("\n📊 [FlowScore 4.0 핵심 변수 Top 5]")
print(imp.head(5))

# 저장
joblib.dump(model, 'FlowScore_v4_CTGAN_Model.pkl')
print("\n💾 모델 저장 완료: FlowScore_v4_CTGAN_Model.pkl")

🚀 [Step 1] 데이터 병합 및 시드 데이터셋 구성 중...
✅ 병합 완료: 총 631개 기업 데이터 확보

🚀 [Step 2] CTGAN 학습 시작 (AI가 부실 패턴을 학습합니다)...


Gen. (-1.08) | Discrim. (-0.12): 100%|██████████| 300/300 [00:29<00:00, 10.15it/s]


✅ CTGAN 학습 완료!
🏭 AI가 새로운 부실 기업 1000건 생성 중...

🚀 [Step 3] FlowScore 4.0 최종 모델 학습...
✅ 최종 모델 정확도: 99.56%

📊 [FlowScore 4.0 핵심 변수 Top 5]
             Feature  Importance
5          CEO_Score    0.344973
10  Weekend_Tx_Ratio    0.237246
9      Tx_Volatility    0.117837
8        Rev_Per_Emp    0.089268
11       Avg_Tx_Hour    0.044632

💾 모델 저장 완료: FlowScore_v4_CTGAN_Model.pkl
